In [1]:
import os 
import shutil
original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './splitted'

In [5]:
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('validation size(',clss,'):', len(train_fnames))
    for fname in validation_fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src,dst)

    test_fnames = fnames[(train_size + validation_size):
                         (validation_size + train_size + test_size)]
    print('test size(',clss,'):', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(os.path.join(test_dir, clss),fname)
        shutil.copyfile(src,dst)


Train size( Apple___Apple_scab ):  378
validation size( Apple___Apple_scab ): 378
test size( Apple___Apple_scab ): 126
Train size( Apple___Black_rot ):  372
validation size( Apple___Black_rot ): 372
test size( Apple___Black_rot ): 124
Train size( Apple___Cedar_apple_rust ):  165
validation size( Apple___Cedar_apple_rust ): 165
test size( Apple___Cedar_apple_rust ): 55
Train size( Apple___healthy ):  987
validation size( Apple___healthy ): 987
test size( Apple___healthy ): 329
Train size( Cherry___healthy ):  512
validation size( Cherry___healthy ): 512
test size( Cherry___healthy ): 170
Train size( Cherry___Powdery_mildew ):  631
validation size( Cherry___Powdery_mildew ): 631
test size( Cherry___Powdery_mildew ): 210
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  307
validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ): 307
test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ): 102
Train size( Corn___Common_rust ):  715
validation size( Corn___Common_rust ): 715

In [3]:
print(DEVICE)

cuda


In [2]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 30 

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train',
                            transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val',
                          transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE, shuffle=True,
                                           num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)


In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):

        super(Net,self).__init__()

        self.conv1 = nn.Conv2d(3,32,3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3, padding=1)
        self.conv3 = nn.Conv2d(64,64,3, padding=1)

        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512,33)

    def forward(self,x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training = self.training)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x,p=0.25, training = self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training = self.training)

        x = x.view(-1, 4096) # flatten with feature map
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training = self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr= 0.001)


In [5]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [10]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0 
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print(' ------ epoch{} ------'.format(epoch))
        print('train_loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val_loss : {:.4f}, Accuracy: {:2f}%'.format(val_loss, val_acc))
        print('completed in {:0f}m{:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)

torch.save(base,'baseline.pt')

 ------ epoch1 ------
train_loss: 0.7636, Accuracy: 77.13%
val_loss : 0.8148, Accuracy: 75.366128%
completed in 1.000000m23.431010s
 ------ epoch2 ------
train_loss: 0.5728, Accuracy: 82.70%
val_loss : 0.6329, Accuracy: 80.836150%
completed in 1.000000m8.612334s
 ------ epoch3 ------
train_loss: 0.5058, Accuracy: 84.16%
val_loss : 0.5719, Accuracy: 82.238077%
completed in 1.000000m6.470571s
 ------ epoch4 ------
train_loss: 0.4223, Accuracy: 87.06%
val_loss : 0.4927, Accuracy: 84.353486%
completed in 1.000000m5.037770s
 ------ epoch5 ------
train_loss: 0.3611, Accuracy: 88.78%
val_loss : 0.4398, Accuracy: 86.206033%
completed in 1.000000m2.870149s
 ------ epoch6 ------
train_loss: 0.3114, Accuracy: 90.71%
val_loss : 0.3958, Accuracy: 87.645513%
completed in 1.000000m2.964771s
 ------ epoch7 ------
train_loss: 0.3249, Accuracy: 90.58%
val_loss : 0.4122, Accuracy: 86.681687%
completed in 1.000000m2.800774s
 ------ epoch8 ------
train_loss: 0.2606, Accuracy: 92.48%
val_loss : 0.3502, Accu

In [ ]:
# Practiced with Resource from 
# book: "한 줄씩 따라 해보는 파이토치 딥러닝 프로젝트 모음집" by 이경택(KyungTaek Lee)
# book's link: https://www.yes24.com/Product/Goods/102911732
# data's link: https://data.mendeley.com/datasets/tywbtsjrjv/1
